In [1]:
print("ok")

ok


In [2]:


from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import os


In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyMuPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#extracted_data

In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 4373


In [8]:
#text_chunks

In [7]:

#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

In [9]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [10]:
print(query_result)

[-0.03447723761200905, 0.031023213639855385, 0.006734990980476141, 0.02610895223915577, -0.03936200216412544, -0.16030248999595642, 0.06692393124103546, -0.006441502831876278, -0.04745049029588699, 0.014758865348994732, 0.07087529450654984, 0.05552753433585167, 0.019193345680832863, -0.026251327246427536, -0.010109513066709042, -0.026940496638417244, 0.022307435050606728, -0.022226642817258835, -0.1496925801038742, -0.01749304123222828, 0.007676258217543364, 0.05435232073068619, 0.0032544711139053106, 0.031725890934467316, -0.0846213549375534, -0.029405983164906502, 0.05159558728337288, 0.04812406003475189, -0.0033148040529340506, -0.05827920883893967, 0.04196924716234207, 0.022210638970136642, 0.1281888484954834, -0.022338991984725, -0.011656233109533787, 0.06292837113142014, -0.03287629410624504, -0.09122603386640549, -0.031175388023257256, 0.052699580788612366, 0.0470348484814167, -0.08420310914516449, -0.030056176707148552, -0.020744847133755684, 0.009517889469861984, -0.0037218490

In [11]:
DB_FAISS_PATH = "vectorstore/db_faiss"

# Create Folder if not exists
os.makedirs(DB_FAISS_PATH, exist_ok=True)

try:
    print(f"Number of Chunks: {len(text_chunks)}")

    print("Creating FAISS Vector Store...")

    db = FAISS.from_documents(text_chunks, embeddings)
    
    # Save FAISS DB Locally
    db.save_local(DB_FAISS_PATH)
    print("FAISS Vector Store Created Successfully")

except Exception as e:
    print("Error while creating FAISS Vector Store:", str(e))


Number of Chunks: 4373
Creating FAISS Vector Store...


KeyboardInterrupt: 

In [12]:
from langchain.vectorstores import FAISS

# Load FAISS Vector Store 
db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)

query = "what might be the reasons for fever?"
docs = db.similarity_search(query, k=2)

print("Relevant Documents Found:")
for i, doc in enumerate(docs):
    print(f"\nResult {i+1}:")
    print(doc.page_content)
    print("Source:", doc.metadata["source"])


Relevant Documents Found:

Result 1:
Fever is frequently associated with chills, sweating, headache, muscular ache, malaise, anorexia or nausea. In children,
fever may be associated with abdominal pain, diarrhoea and vomiting. Mild to moderate anaemia is frequent in children
and pregnant women.
Severe malaria
Source: data\diease detector 1.pdf

Result 2:
Signs of severity
Infectious causes of fever according to signs and symptoms
Petechial or purpuric rash, meningeal signs, heart murmur, severe abdominal pain, dehydration.
Signs of severe bacterial infection or sepsis: critically ill appearance , hypothermia, altered level of consciousness,
Source: data\diease detector 1.pdf


In [13]:

prompt_template = """
Welcome to Med Guide 🩺💬,
Your AI-Powered Symptom Checker & Initial Guide.

Based on the symptoms you mentioned, here's what I found:

Use the following pieces of information to **provide a possible diagnosis** and guidance based on the symptoms.
If you're not sure, give the **most likely health condition**.

Context: {context}
User Symptoms: {question}

If no relevant information is found, say: **"I couldn't find exact information, but it's better to consult a doctor."**

Helpful Answer:
"""


In [21]:
from langchain_huggingface import HuggingFaceEndpoint


# Step 1: Setup LLM (Mistral with HuggingFace)
HF_TOKEN=os.environ.get("HF_TOKEN")
HUGGINGFACE_REPO_ID="mistralai/Mistral-7B-Instruct-v0.3"

def load_llm(huggingface_repo_id):
    llm=HuggingFaceEndpoint(
        repo_id=huggingface_repo_id,
        temperature=0.2,
        model_kwargs={"token":HF_TOKEN,
                      "max_length":"512"}
    )
    return llm

In [22]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [23]:

# Create QA chain
qa_chain=RetrievalQA.from_chain_type(
    llm=load_llm(HUGGINGFACE_REPO_ID),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [24]:
# Now invoke with a single query
user_query=input("Write Query Here: ")
response=qa_chain.invoke({'query': user_query})
print("RESULT: ", response["result"])
print("SOURCE DOCUMENTS: ", response["source_documents"])

c:\Users\monic\anaconda3\envs\medguide\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


RESULT:  
Based on the symptoms you provided, it seems like you might be experiencing symptoms that could be associated with a respiratory infection, such as pneumonia or bronchitis. However, it's also possible that these symptoms could be indicative of other conditions like chronic obstructive pulmonary disease (COPD) or even lung cancer.

Given the prolonged nature of your cough, it's important to consult a healthcare professional for a proper diagnosis. They can perform tests such as a chest X-ray or CT scan to help determine the cause of your symptoms.

In the meantime, try to rest, stay hydrated, and avoid smoking or exposure to secondhand smoke. If you experience severe chest pain, difficulty breathing, or fever, seek immediate medical attention.

If you're experiencing a coughing fit followed by a distinctive gasping sound and/or post-tussive vomiting, this could potentially indicate whooping cough (pertussis). This is a highly contagious bacterial infection that primarily affec